do pliku `AndroidManifest.xml`
```xml
<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools">

    <uses-permission android:name="android.permission.INTERNET" />

    <application
        android:name=".NewsApplication"
        android:allowBackup="true"
        android:dataExtractionRules="@xml/data_extraction_rules"
        android:fullBackupContent="@xml/backup_rules"
        ...
    </application>

</manifest>
```

do pliku `build.gradle(Project)`

```kotlin
plugins {
    alias(libs.plugins.android.application) apply false
    alias(libs.plugins.kotlin.android) apply false
    alias(libs.plugins.kotlin.compose) apply false
    id("com.google.dagger.hilt.android") version "2.57.2" apply false
    id("androidx.room") version "2.8.1" apply false
    id("com.google.devtools.ksp") version "2.0.21-1.0.27" apply false
}
```

do pliku `build.gradle(Module)`

```kotlin
plugins {
    alias(libs.plugins.android.application)
    alias(libs.plugins.kotlin.android)
    alias(libs.plugins.kotlin.compose)
    id("com.google.devtools.ksp")
    id("com.google.dagger.hilt.android")
    id("androidx.room")
}

android {
    ...
    room {
        schemaDirectory("$projectDir/schemas")
    }
}

dependencies {

    implementation("com.google.dagger:hilt-android:2.57.2")
    ksp("com.google.dagger:hilt-android-compiler:2.57.2")

    // Retrofit do komunikacji sieciowej
    implementation("com.squareup.retrofit2:retrofit:3.0.0")
    // Konwerter GSON do parsowania JSON
    implementation("com.squareup.retrofit2:converter-gson:3.0.0")

    // ViewModel i Lifecycle w Compose
    implementation("androidx.lifecycle:lifecycle-viewmodel-compose:2.9.4")
    implementation("androidx.lifecycle:lifecycle-runtime-compose:2.9.4")
    implementation("androidx.hilt:hilt-navigation-compose:1.3.0")

    implementation("androidx.room:room-runtime:2.8.1")
    ksp("androidx.room:room-compiler:2.8.1")
    implementation("androidx.room:room-ktx:2.8.1")

    implementation("androidx.navigation:navigation-compose:2.9.5")

    ...
}
```

```kotlin
// --- Konfiguracja Hilt ---
@HiltAndroidApp
class NewsApplication : Application()

// --- Warstwa Danych: Sieć (Retrofit) ---
data class NewsApiResponse(val articles: List<ArticleDto>)
data class ArticleDto(val title: String?, val description: String?)
interface ApiService {
    @GET("v2/top-headlines?country=us&category=technology")
    suspend fun getTopHeadlines
                (@Header("X-Api-Key") apiKey: String): NewsApiResponse
}

// --- Warstwa Danych: Baza Danych (Room) ---
@Entity(tableName = "articles")
data class ArticleEntity(
    @PrimaryKey val title: String,
    val description: String?
)
@Dao
interface ArticleDao {
    @Query("SELECT * FROM articles")
    fun getArticlesStream(): Flow<List<ArticleEntity>>
    @Query("SELECT * FROM articles WHERE title = :title")
    fun getArticleByTitle(title: String): Flow<ArticleEntity?>
    @Upsert
    suspend fun insertArticles(articles: List<ArticleEntity>)
    @Query("DELETE FROM articles")
    suspend fun clearAll()
}
@Database(entities = [ArticleEntity::class], version = 1, exportSchema = false)
abstract class AppDatabase : RoomDatabase() {
    abstract fun articleDao(): ArticleDao
}

// --- Warstwa Danych: Repozytorium ---
class ArticleRepository @Inject constructor(
    private val apiService: ApiService,
    private val articleDao: ArticleDao
) {
    val articlesStream: Flow<List<ArticleEntity>> =
        articleDao.getArticlesStream()
    fun getArticleByTitle(title: String): Flow<ArticleEntity?> =
        articleDao.getArticleByTitle(title)
    suspend fun refreshNews() {
        try {
            val apiKey = "twój_klucz_api"
            val response = apiService.getTopHeadlines(apiKey)
            val entities = response.articles
                .filter { it.title != null && it.description != null }
                .map { ArticleEntity(it.title!!, it.description) }

            articleDao.clearAll()
            articleDao.insertArticles(entities)
        } catch (e: Exception) {
            Log.e("ArticleRepository", "Failed to fetch news", e)
        }
    }
}

// --- Moduł Hilt (Wstrzykiwanie Zależności) ---
@Module
@InstallIn(SingletonComponent::class)
object AppModule {
    @Provides
    @Singleton
    fun provideAppDatabase(@ApplicationContext context: Context): AppDatabase {
        return Room.databaseBuilder(context, AppDatabase::class.java, "news_database_db").build()
    }
    @Provides
    @Singleton
    fun provideArticleDao(database: AppDatabase): ArticleDao = database.articleDao()
    @Provides
    @Singleton
    fun provideRetrofit(): Retrofit {
        return Retrofit.Builder()
            .baseUrl("https://newsapi.org/")
            .addConverterFactory(GsonConverterFactory.create())
            .build()
    }
    @Provides
    @Singleton
    fun provideApiService(retrofit: Retrofit): ApiService = retrofit.create(ApiService::class.java)
}

// --- Architektura: ViewModel i Stan (Ekran Listy) ---
data class ArticlesUiState(
    val articles: List<ArticleEntity> = emptyList(),
    val isLoading: Boolean = false
)

@HiltViewModel
class ArticlesViewModel @Inject constructor(
    private val repository: ArticleRepository
) : ViewModel() {
    private val _isLoading = MutableStateFlow(false)
    val uiState: StateFlow<ArticlesUiState> = combine(
        repository.articlesStream,
        _isLoading
    ) { articles, isLoading ->
        ArticlesUiState(articles, isLoading)
    }
        .stateIn(
        scope = viewModelScope,
        started = SharingStarted.WhileSubscribed(5000),
        initialValue = ArticlesUiState(isLoading = true)
    )

    init {
        refresh()
    }
    fun refresh() {
        viewModelScope.launch {
            _isLoading.value = true
            repository.refreshNews()
            _isLoading.value = false
        }
    }
}

data class ArticleDetailState(
    val article: ArticleEntity? = null,
    val isLoading: Boolean = true
)

@HiltViewModel
class ArticleDetailViewModel @Inject constructor(
    repository: ArticleRepository,
    savedStateHandle: SavedStateHandle
) : ViewModel() {
    private val articleTitle: String =
        savedStateHandle.get<String>("articleTitle") ?: ""
    val uiState: StateFlow<ArticleDetailState> = repository
        .getArticleByTitle(Uri.decode(articleTitle))
        .map { article ->
            ArticleDetailState(article = article, isLoading = false)
        }
        .stateIn(
            scope = viewModelScope,
            started = SharingStarted.WhileSubscribed(5000),
            initialValue = ArticleDetailState(isLoading = true)
        )
}


// --- UI: Nawigacja i Ekrany ---
@AndroidEntryPoint
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        enableEdgeToEdge()
        setContent {
            MaterialTheme {
                AppNavigation()
            }
        }
    }
}

@Composable
fun AppNavigation() {
    val navController = rememberNavController()
    NavHost(navController = navController, startDestination = "list") {
        composable("list") {
            ArticleListScreen(onArticleClick = { title ->
                navController.navigate("detail/${Uri.encode(title)}")
            })
        }
        composable(
            "detail/{articleTitle}",
            arguments = listOf(navArgument("articleTitle") { type = NavType.StringType })
        ) {
            ArticleDetailScreen()
        }
    }
}

@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun ArticleListScreen(
    viewModel: ArticlesViewModel = hiltViewModel(),
    onArticleClick: (String) -> Unit
) {
    val uiState by viewModel.uiState.collectAsStateWithLifecycle()

    Scaffold(topBar = { TopAppBar(title = { Text("Wiadomości Technologiczne") }) }) { padding ->
        Box(modifier = Modifier.fillMaxSize().padding(padding)) {
            if (uiState.isLoading && uiState.articles.isEmpty()) {
                CircularProgressIndicator(modifier = Modifier.align(Alignment.Center))
            } else {
                LazyColumn(contentPadding = PaddingValues(16.dp)) {
                    items(uiState.articles) { article ->
                        ArticleItem(article = article, onClick = { onArticleClick(article.title) })
                        Spacer(modifier = Modifier.height(12.dp))
                    }
                }
            }
        }
    }
}

@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun ArticleDetailScreen(viewModel: ArticleDetailViewModel = hiltViewModel()) {
    val uiState by viewModel.uiState.collectAsStateWithLifecycle()

    Scaffold(topBar = { TopAppBar(title = { Text("Szczegóły Artykułu") }) }) { padding ->
        Box(
            modifier = Modifier.fillMaxSize().padding(padding),
            contentAlignment = Alignment.Center
        ) {
            if (uiState.isLoading) {
                CircularProgressIndicator()
            } else if (uiState.article == null) {
                Text("Nie znaleziono artykułu.")
            } else {
                val article = uiState.article!!
                LazyColumn(modifier = Modifier.fillMaxSize(), contentPadding = PaddingValues(16.dp)) {
                    item {
                        Text(article.title, style = MaterialTheme.typography.headlineMedium, fontWeight = FontWeight.Bold)
                        Spacer(Modifier.height(16.dp))
                        Text(article.description ?: "Brak opisu.", style = MaterialTheme.typography.bodyLarge)
                    }
                }
            }
        }
    }
}

@Composable
fun ArticleItem(article: ArticleEntity, onClick: () -> Unit) {
    Card(modifier = Modifier.clickable(onClick = onClick), elevation = CardDefaults.cardElevation(4.dp)) {
        Column(modifier = Modifier.padding(16.dp)) {
            Text(article.title, style = MaterialTheme.typography.titleMedium, fontWeight = FontWeight.Bold)
        }
    }
}
```